In [0]:
#Import necessary libraries required

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
#Checking if both files are present

In [0]:
%fs ls /FileStore/tables/project/

path,name,size,modificationTime
dbfs:/FileStore/tables/project/customers.json,customers.json,15815082,1660644811000
dbfs:/FileStore/tables/project/orders.csv,orders.csv,32138462,1660644924000
dbfs:/FileStore/tables/project/with_partitions/,with_partitions/,0,0
dbfs:/FileStore/tables/project/without_partitions/,without_partitions/,0,0


In [0]:
#Reading JSON file as customers

In [0]:
customers = spark.read.json("/FileStore/tables/project/customers.json")

In [0]:
customers.show(10)

+-----------+-----------+--------------+--------------------+------+-----------+---------+-----+-----+----+-------+-----------------+
|       City|     County|Customer Since|              E Mail|Gender| Place Name|   Region|State|  Zip| age|cust_id|        full_name|
+-----------+-----------+--------------+--------------------+------+-----------+---------+-----+-----+----+-------+-----------------+
|     Vinson|     Harmon|     8/22/2006|jani.titus@gmail.com|     F|     Vinson|    South|   OK|73571|43.0|60124.0|      Titus, Jani|
|     Graham|   Bradford|      2/4/1981| lee.eaker@gmail.com|     M|     Graham|    South|   FL|32042|28.0|42485.0|       Eaker, Lee|
|Grand Forks|Grand Forks|     6/27/2010|jason.simoneau@gm...|     M|Grand Forks|  Midwest|   ND|58201|65.0|53620.0|  Simoneau, Jason|
|Laupahoehoe|     Hawaii|      4/3/1992|grover.bayless@ya...|     M|Laupahoehoe|     West|   HI|96764|33.0|56836.0|  Bayless, Grover|
|     Glendo|     Platte|     6/21/2015|albertina.bensen@...| 

In [0]:
# Changing schema to convert it proper datataype

In [0]:
customers_schema = StructType([StructField('City', StringType(), True),
                     StructField('County', StringType(), True),
                     StructField('Customer Since', StringType(), True),
                     StructField('E Mail', StringType(), True),                  
                     StructField('Gender', StringType(), True),      
                     StructField('Place Name', StringType(), True),
                     StructField('Region', StringType(), True),
                     StructField('State', StringType(), True),
                     StructField('Zip', IntegerType(), True),       
                     StructField('Age', FloatType(), True),       
                     StructField('cust_id', DoubleType(), True),       
                     StructField('full_name', StringType(), True)               
                     ])

In [0]:
customers.printSchema()

root
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Customer Since: string (nullable = true)
 |-- E Mail: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Place Name: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip: long (nullable = true)
 |-- age: double (nullable = true)
 |-- cust_id: double (nullable = true)
 |-- full_name: string (nullable = true)



In [0]:

#Loading orders csv with default schema

In [0]:
orders_df=spark.read.csv("/FileStore/tables/project/orders.csv",header=True,inferSchema=True)

In [0]:
orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- item_id: double (nullable = true)
 |-- qty_ordered: double (nullable = true)
 |-- price: double (nullable = true)
 |-- value: double (nullable = true)
 |-- discount_amount: double (nullable = true)
 |-- total: double (nullable = true)
 |-- category: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- cust_id: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)



In [0]:
display(orders_df.limit(5))

order_id,order_date,status,item_id,qty_ordered,price,value,discount_amount,total,category,payment_method,cust_id,year,month
100354678,2020-10-01T00:00:00.000+0000,received,574772.0,21.0,89.9,1798.0,0.0,1798.0,Men's Fashion,cod,60124.0,2020,Oct-2020
100354678,2020-10-01T00:00:00.000+0000,received,574774.0,11.0,19.0,190.0,0.0,190.0,Men's Fashion,cod,60124.0,2020,Oct-2020
100354680,2020-10-01T00:00:00.000+0000,complete,574777.0,9.0,149.9,1199.2,0.0,1199.2,Men's Fashion,cod,60124.0,2020,Oct-2020
100354680,2020-10-01T00:00:00.000+0000,complete,574779.0,9.0,79.9,639.2,0.0,639.2,Men's Fashion,cod,60124.0,2020,Oct-2020
100367357,2020-11-13T00:00:00.000+0000,received,595185.0,2.0,99.9,99.9,0.0,99.9,Men's Fashion,cod,60124.0,2020,Nov-2020


In [0]:
#Combining customers JSON and orders CSV file by inner join and dropping customers cust_id column

In [0]:
combine=orders_df.join(customers,orders_df.cust_id==customers.cust_id,"inner").drop(customers.cust_id)

In [0]:
combine.show(10)

+---------+-------------------+--------+--------+-----------+------+------+---------------+------+-----------------+--------------+-------+----+--------+------------+--------+--------------+--------------------+------+------------+---------+-----+-----+----+-------------------+
| order_id|         order_date|  status| item_id|qty_ordered| price| value|discount_amount| total|         category|payment_method|cust_id|year|   month|        City|  County|Customer Since|              E Mail|Gender|  Place Name|   Region|State|  Zip| age|          full_name|
+---------+-------------------+--------+--------+-----------+------+------+---------------+------+-----------------+--------------+-------+----+--------+------------+--------+--------------+--------------------+------+------------+---------+-----+-----+----+-------------------+
|100354830|2020-10-01 00:00:00|complete|575059.0|        5.0| 454.3|1817.2|          800.0|1017.2|  Women's Fashion|           cod|56899.0|2020|Oct-2020|South Denn

In [0]:
combine.createOrReplaceTempView('combined')

In [0]:
%sql

select * from combined limit 10

order_id,order_date,status,item_id,qty_ordered,price,value,discount_amount,total,category,payment_method,cust_id,year,month,City,County,Customer Since,E Mail,Gender,Place Name,Region,State,Zip,age,full_name
100354830,2020-10-01T00:00:00.000+0000,complete,575059.0,5.0,454.3,1817.2,800.0,1017.2,Women's Fashion,cod,56899.0,2020,Oct-2020,South Dennis,Cape May,7/30/1993,kendrick.stacks@hotmail.com,M,South Dennis,Northeast,NJ,8245,38.0,"Stacks, Kendrick"
100355727,2020-10-06T00:00:00.000+0000,complete,576680.0,2.0,101.7,101.7,0.0,101.7,Mobiles & Tablets,cod,41877.0,2020,Oct-2020,Pinckard,Dale,1/2/2003,harriett.mcsweeney@hotmail.co.uk,F,Pinckard,South,AL,36371,29.0,"Mcsweeney, Harriett"
100537859,2021-07-04T00:00:00.000+0000,refund,864119.0,2.0,160.0,160.0,0.0,160.0,Mobiles & Tablets,cod,41877.0,2021,Jul-2021,Pinckard,Dale,1/2/2003,harriett.mcsweeney@hotmail.co.uk,F,Pinckard,South,AL,36371,29.0,"Mcsweeney, Harriett"
100537859,2021-07-04T00:00:00.000+0000,refund,864120.0,2.0,44.5,44.5,0.0,44.5,Computing,cod,41877.0,2021,Jul-2021,Pinckard,Dale,1/2/2003,harriett.mcsweeney@hotmail.co.uk,F,Pinckard,South,AL,36371,29.0,"Mcsweeney, Harriett"
100537859,2021-07-04T00:00:00.000+0000,refund,864121.0,2.0,47.405,47.405,0.0,47.405,Computing,cod,41877.0,2021,Jul-2021,Pinckard,Dale,1/2/2003,harriett.mcsweeney@hotmail.co.uk,F,Pinckard,South,AL,36371,29.0,"Mcsweeney, Harriett"
100537859,2021-07-04T00:00:00.000+0000,refund,864122.0,2.0,19.5,19.5,0.0,19.5,Beauty & Grooming,cod,41877.0,2021,Jul-2021,Pinckard,Dale,1/2/2003,harriett.mcsweeney@hotmail.co.uk,F,Pinckard,South,AL,36371,29.0,"Mcsweeney, Harriett"
100537859,2021-07-04T00:00:00.000+0000,refund,864123.0,2.0,64.9,64.9,0.0,64.9,Mobiles & Tablets,cod,41877.0,2021,Jul-2021,Pinckard,Dale,1/2/2003,harriett.mcsweeney@hotmail.co.uk,F,Pinckard,South,AL,36371,29.0,"Mcsweeney, Harriett"
100537859,2021-07-04T00:00:00.000+0000,refund,864124.0,3.0,21.0,42.0,0.0,42.0,Home & Living,cod,41877.0,2021,Jul-2021,Pinckard,Dale,1/2/2003,harriett.mcsweeney@hotmail.co.uk,F,Pinckard,South,AL,36371,29.0,"Mcsweeney, Harriett"
100537859,2021-07-04T00:00:00.000+0000,refund,864125.0,2.0,14.5,14.5,0.0,14.5,Beauty & Grooming,cod,41877.0,2021,Jul-2021,Pinckard,Dale,1/2/2003,harriett.mcsweeney@hotmail.co.uk,F,Pinckard,South,AL,36371,29.0,"Mcsweeney, Harriett"
100560972,2021-09-20T00:00:00.000+0000,received,902624.0,2.0,39.9,39.9,0.0,39.9,Computing,cod,41877.0,2021,Sep-2021,Pinckard,Dale,1/2/2003,harriett.mcsweeney@hotmail.co.uk,F,Pinckard,South,AL,36371,29.0,"Mcsweeney, Harriett"


In [0]:
%sql
describe without_partitions

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table or view not found: without_partitions; line 1 pos 9;
'DescribeRelation false, [col_name#571, data_type#572, comment#573]
+- 'UnresolvedTableOrView [without_partitions], DESCRIBE TABLE, true

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2(CheckAnalysis.scala:131)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2$adapted(CheckAnalysis.scala:105)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:358)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:105)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:100)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:100)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:222)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:276)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:331)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:273)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:128)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:151)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:265)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:265)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:129)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:126)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:118)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:103)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:803)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:798)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:695)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.immutable.List.map(List.scala:293)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:145)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:605)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4

In [0]:
%sql
select category, count(status) from without_partitions where year="2020" and status="refund"
group by category
order by count(status) desc

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table or view not found: without_partitions; line 1 pos 36;
'Sort ['count('status) DESC NULLS LAST], true
+- 'Aggregate ['category], ['category, unresolvedalias('count('status), None)]
 +- 'Filter (('year = 2020) AND ('status = refund))
 +- 'UnresolvedRelation [without_partitions], [], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2(CheckAnalysis.scala:138)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$2$adapted(CheckAnalysis.scala:105)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:358)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:357)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:357)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:105)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:100)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:100)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:222)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:276)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:331)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:273)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:128)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:151)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:265)
	at org.apache.spark.sql.SparkSession.withActiv

In [0]:
#Converting it into parquet file without partitions and store only one parquet file

In [0]:
combine.repartition(1).write.parquet("/FileStore/tables/project/without_partitions")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-4040562479432674> in <module>
----> 1 combine.repartition(1).write.parquet("/FileStore/tables/project/without_partitions")

/databricks/spark/python/pyspark/sql/readwriter.py in parquet(self, path, mode, partitionBy, compression)
    883             self.partitionBy(partitionBy)
    884         self._set_opts(compression=compression)
--> 885         self._jwrite.parquet(path)
    886 
    887     def text(self, path, compression=None, lineSep=None):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    121     

In [0]:
#check if parquet file is present in the folder

In [0]:
%fs ls /FileStore/tables/project/without_partitions

path,name,size,modificationTime
dbfs:/FileStore/tables/project/without_partitions/_SUCCESS,_SUCCESS,0,1660992658000
dbfs:/FileStore/tables/project/without_partitions/_committed_1548538402718866121,_committed_1548538402718866121,123,1660992657000
dbfs:/FileStore/tables/project/without_partitions/_started_1548538402718866121,_started_1548538402718866121,0,1660992648000
dbfs:/FileStore/tables/project/without_partitions/part-00000-tid-1548538402718866121-6ee17854-bb0f-42c9-81b0-365aa47b02f8-85-1-c000.snappy.parquet,part-00000-tid-1548538402718866121-6ee17854-bb0f-42c9-81b0-365aa47b02f8-85-1-c000.snappy.parquet,10693557,1660992656000


In [0]:
#Converting it into parquet file with partitions for year

In [0]:
combine.write.partitionBy("year").parquet("/FileStore/tables/project/with_partitions/year")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-4040562479432678> in <module>
----> 1 combine.write.partitionBy("year").parquet("/FileStore/tables/project/with_partitions/year")

/databricks/spark/python/pyspark/sql/readwriter.py in parquet(self, path, mode, partitionBy, compression)
    883             self.partitionBy(partitionBy)
    884         self._set_opts(compression=compression)
--> 885         self._jwrite.parquet(path)
    886 
    887     def text(self, path, compression=None, lineSep=None):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    1

In [0]:
%fs ls /FileStore/tables/project/with_partitions/year

path,name,size,modificationTime
dbfs:/FileStore/tables/project/with_partitions/year/_SUCCESS,_SUCCESS,0,1660992690000
dbfs:/FileStore/tables/project/with_partitions/year/year=2020/,year=2020/,0,0
dbfs:/FileStore/tables/project/with_partitions/year/year=2021/,year=2021/,0,0


In [0]:
#Converting it into parquet file with partitions for month

In [0]:
combine.write.partitionBy("month").parquet("/FileStore/tables/project/with_partitions/month")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-4040562479432681> in <module>
----> 1 combine.write.partitionBy("month").parquet("/FileStore/tables/project/with_partitions/month")

/databricks/spark/python/pyspark/sql/readwriter.py in parquet(self, path, mode, partitionBy, compression)
    883             self.partitionBy(partitionBy)
    884         self._set_opts(compression=compression)
--> 885         self._jwrite.parquet(path)
    886 
    887     def text(self, path, compression=None, lineSep=None):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
   

In [0]:
%fs ls /FileStore/tables/project/with_partitions/month

path,name,size,modificationTime
dbfs:/FileStore/tables/project/with_partitions/month/_SUCCESS,_SUCCESS,0,1660992720000
dbfs:/FileStore/tables/project/with_partitions/month/month=Apr-2021/,month=Apr-2021/,0,0
dbfs:/FileStore/tables/project/with_partitions/month/month=Aug-2021/,month=Aug-2021/,0,0
dbfs:/FileStore/tables/project/with_partitions/month/month=Dec-2020/,month=Dec-2020/,0,0
dbfs:/FileStore/tables/project/with_partitions/month/month=Feb-2021/,month=Feb-2021/,0,0
dbfs:/FileStore/tables/project/with_partitions/month/month=Jan-2021/,month=Jan-2021/,0,0
dbfs:/FileStore/tables/project/with_partitions/month/month=Jul-2021/,month=Jul-2021/,0,0
dbfs:/FileStore/tables/project/with_partitions/month/month=Jun-2021/,month=Jun-2021/,0,0
dbfs:/FileStore/tables/project/with_partitions/month/month=Mar-2021/,month=Mar-2021/,0,0
dbfs:/FileStore/tables/project/with_partitions/month/month=May-2021/,month=May-2021/,0,0


In [0]:
#1

In [0]:
#Creating a table from November-2020 month parquet file and storing it in memory for writing SQL queries for question 1

In [0]:
%sql 
CREATE TABLE with_partitions_nov2020
USING parquet
OPTIONS (path "/FileStore/tables/project/with_partitions/month/month=Nov-2020")

In [0]:
%sql 
select category,sum(total)as Revenue from with_partitions_nov2020
where status='complete'
group by category

category,Revenue
School & Education,1621.6419999999998
Entertainment,588682.5740799999
Kids & Baby,6397.602190000001
Computing,143988.45747999984
Mobiles & Tablets,1372377.362470003
Beauty & Grooming,66664.34888000002
Health & Sports,8903.248259999997
Soghaat,10498.116670000005
Books,276.3
Others,151800.0


In [0]:
#Creating a table from year 2020 parquet file and storing it in memory for writing SQL queries for question 2

In [0]:
%sql 
CREATE TABLE with_partitions_year2020
USING parquet
OPTIONS (path "/FileStore/tables/project/with_partitions/year/year=2020")

In [0]:
#2

In [0]:
%sql
select category, count(status) as Refunds from with_partitions_year2020 
where status="order_refunded"
group by category
order by count(status) desc limit 5

category,Refunds
Men's Fashion,3387
Mobiles & Tablets,1888
Women's Fashion,1472
Appliances,894
Beauty & Grooming,613


In [0]:
#Creating a table from without partitions parquet file and storing it in memory for writing SQL queries for question 3

In [0]:
%sql 
CREATE TABLE without_partitions
USING parquet
OPTIONS (path "/FileStore/tables/project/without_partitions")

In [0]:
#3

In [0]:
%sql
select  year,month,category, count( distinct order_id)as orders from without_partitions
where status='complete'
group by category,year,month
order by year,month,category

year,month,category,orders
2020,Dec-2020,Appliances,6644
2020,Dec-2020,Beauty & Grooming,1331
2020,Dec-2020,Books,33
2020,Dec-2020,Computing,1094
2020,Dec-2020,Entertainment,3001
2020,Dec-2020,Health & Sports,334
2020,Dec-2020,Home & Living,1314
2020,Dec-2020,Kids & Baby,440
2020,Dec-2020,Men's Fashion,4587
2020,Dec-2020,Mobiles & Tablets,9650


In [0]:
%sql
 
with cte(category,Young,Adult,Middle,Old) as (select category
,case when age between 0 and 20 then 'Young' end as Young
,case when age between 21 and 35 then 'Adult' end as Adult
,case when age between 36 and 54 then 'Middle' end as Middle
,case when age>55 then 'Old'  end as Old 
from  without_partitions)
select * from cte

category,Young,Adult,Middle,Old
Women's Fashion,null,null,Middle,null
Mobiles & Tablets,null,Adult,null,null
Mobiles & Tablets,null,Adult,null,null
Computing,null,Adult,null,null
Computing,null,Adult,null,null
Beauty & Grooming,null,Adult,null,null
Mobiles & Tablets,null,Adult,null,null
Home & Living,null,Adult,null,null
Beauty & Grooming,null,Adult,null,null
Computing,null,Adult,null,null


In [0]:
#4

In [0]:
pivot(
sum(total) for age in (Young,Adult,Middle,Old)
) as Pivot

In [0]:
%sql
 select Category,round((Young+Adult+Middle+Old),2)as Total_Spends,round(((Young/(Young+Adult+Middle+Old))*100),2)as Young,round(((Adult/(Young+Adult+Middle+Old))*100),2)as Adult,round(((Middle/(Young+Adult+Middle+Old))*100),2)as Middle,round(((Old/(Young+Adult+Middle+Old))*100),2)as Old from (select category,total,
case
when age between 0 and 20 then 'Young'
when age between 21 and 35 then 'Adult'
when age between 36 and 55 then 'Middle'
Else 'Old' end as Agy 
from  without_partitions
where status='complete')
pivot(
sum(total) for Agy in ('Young','Adult','Middle','Old')
)order by category 


Category,Total_Spends,Young,Adult,Middle,Old
Appliances,1.147006262E7,4.5,27.71,33.35,34.44
Beauty & Grooming,828727.4,3.96,26.17,37.06,32.81
Books,4248.2,3.12,35.69,35.34,25.85
Computing,2375494.2,5.16,30.57,42.29,21.98
Entertainment,1.077237834E7,4.16,27.09,35.57,33.18
Health & Sports,316389.3,3.16,18.77,41.1,36.97
Home & Living,505786.01,5.44,23.89,33.74,36.93
Kids & Baby,237394.29,7.36,20.87,26.6,45.17
Men's Fashion,1415003.21,4.44,26.68,34.69,34.19
Mobiles & Tablets,3.115065927E7,5.34,25.82,34.71,34.13


In [0]:
#5

In [0]:
%sql 

select C.gender,C.category,round((tot/tota)*100,3) as Percentage from 
(select gender,category,sum(total) as tot from without_partitions
where gender="F" and  status='complete'
group by gender,category) C
inner join
(select category,sum(total) as tota from without_partitions
where status='complete'
group by category) D
on C.category=D.category
union
select E.gender,E.category,round((tot/tota)*100,3)  as T from 
(select gender,category,sum(total) as tot from without_partitions
where gender="M" and status='complete'
group by gender,category) E
inner join
(select category,sum(total) as tota from without_partitions
where status='complete'
group by category) F
on E.category=F.category
order by category



gender,category,Percentage
M,Appliances,51.312
F,Appliances,48.688
F,Beauty & Grooming,46.981
M,Beauty & Grooming,53.019
F,Books,40.218
M,Books,59.782
M,Computing,55.76
F,Computing,44.24
F,Entertainment,50.234
M,Entertainment,49.766


In [0]:
#6

In [0]:
%sql

select * from(select rank() over (partition by month order by sum(total)desc) as Ranks,year, month, cust_id, full_name,gender, round(sum(total),3) as Spend from without_partitions
where status='complete'
group by year, month, cust_id, full_name, gender
order by Spend desc) where Ranks<=5
order by month,Ranks


Ranks,year,month,cust_id,full_name,gender,Spend
1,2021,Apr-2021,94842.0,"Banh, Anh",F,63533.816
2,2021,Apr-2021,91986.0,"Bremer, Blake",F,62977.32
3,2021,Apr-2021,94840.0,"Barre, Becki",F,49016.848
4,2021,Apr-2021,89911.0,"Heald, Kathlyn",F,48460.954
5,2021,Apr-2021,97418.0,"Marchant, Thomasina",F,44925.5
1,2021,Aug-2021,3732.0,"Myres, Margherita",F,3046.96
2,2021,Aug-2021,94267.0,"German, Genaro",M,761.84
3,2021,Aug-2021,113303.0,"Bishop, Carrol",F,481.4
4,2021,Aug-2021,46509.0,"Coachman, Lucienne",F,458.9
5,2021,Aug-2021,109646.0,"Yokley, Jackie",F,305.2


In [0]:
#7

In [0]:
%sql
select cust_id, cast(max(order_date)as Date)as last_Transacted_Day,round(datediff('2021-10-31',cast(max(order_date) as Date))/7) as Recency,round(sum(total),2)as Monetary_Value,count(distinct order_id)as Frequency from without_partitions
where status='complete'
group by cust_id 
order by cust_id asc


cust_id,last_Transacted_Day,Recency,Monetary_Value,Frequency
4.0,2021-09-14,7.0,21079.95,14
15.0,2021-02-11,37.0,216.8,3
20.0,2021-04-01,30.0,22379.7,5
21.0,2021-02-03,39.0,105.0,1
28.0,2020-10-17,54.0,70.0,1
32.0,2021-05-28,22.0,44030.98,65
33.0,2021-05-26,23.0,29248.53,38
41.0,2020-11-18,50.0,219.9,1
44.0,2021-04-01,30.0,2237.02,2
47.0,2020-12-31,43.0,12425.01,9


In [0]:
#8

In [0]:
%sql
select cust_id, round(sum(total),2)as Monetary_Value,count(distinct order_id)as Frequency from without_partitions
where status='complete'
group by cust_id 
order by Frequency desc,Monetary_value limit 10

cust_id,Monetary_Value,Frequency
85775.0,29469.4,327
44619.0,227106.79,140
5769.0,125136.23,128
30465.0,134404.33,101
48199.0,71224.29,81
44445.0,57618.93,72
114.0,63527.92,66
44611.0,85164.4,66
32.0,44030.98,65
2478.0,59501.62,65
